In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm

from transformers import AutoProcessor, AutoModelForCausalLM  
from PIL import Image
import requests
import copy
import torch
%matplotlib inline

/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transfor

In [2]:
meld_path = '../data/MELD.Raw'
train_data = pd.read_csv(os.path.join(meld_path, 'test_sent_emo.csv'))
train_data.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378"
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385"
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520"
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146"
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833"


In [3]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/home/mkoltunov/anaconda3/envs/eacl_venv/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [4]:
def run_example(task_prompt, image, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text, 
        task=task_prompt, 
        image_size=(image.width, image.height)
    )

    return parsed_answer

In [ ]:
task_prompt = '<DETAILED_CAPTION>'
visual_desc_train = []

for i, row in tqdm(train_data.iterrows()):
    try:
        video_path = os.path.join(meld_path, "output_repeated_splits_test", f"dia{row['Dialogue_ID']}_utt{row['Utterance_ID']}.mp4")
        vid = cv2.VideoCapture(video_path)
        for _ in range(15):
            ret, frame = vid.read()
        if not ret:
            video_path = os.path.join(meld_path, "output_repeated_splits_test", f"final_videos_testdia{row['Dialogue_ID']}_utt{row['Utterance_ID']}.mp4")
            vid = cv2.VideoCapture(video_path)
            for _ in range(15):
                ret, frame = vid.read()
        
        img = Image.fromarray(frame)
        ans = run_example(task_prompt, img)
        visual_desc_train.append(ans[task_prompt])
    except:
        visual_desc_train.append("Nan")


In [9]:
mod_data = pd.read_csv('./data/MELD/modified_test_data.csv')
mod_data.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,audio_caption
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378",Neutral
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385",The speaker sounds happy.
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520",The speaker sounds like a woman.
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146",The speaker sounds angry.
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833","The speaker sounds male, aged between 16-25 ye..."


In [11]:
mod_data['visual_caption'] = visual_desc_train
mod_data.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,audio_caption,visual_caption
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378",Neutral,The image shows a man and woman standing in a ...
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385",The speaker sounds happy.,The image shows a man and woman standing in a ...
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520",The speaker sounds like a woman.,The image shows a man and woman sitting at a t...
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146",The speaker sounds angry.,The image shows a woman laying in a hospital b...
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833","The speaker sounds male, aged between 16-25 ye...",The image shows a woman laying in a hospital b...


In [12]:
# mod_data.to_csv("./data/MELD/modified_test_data.csv", index=False)